In [14]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras import Input
from keras.utils import to_categorical
import mlflow.keras
import yaml

In [5]:
mlflow.tensorflow.autolog()
annotation_file = "annotations.yaml"
with open(annotation_file, 'r') as file:
    annotations = yaml.safe_load(file)
data_version = annotations["data_version"]
mlflow.log_param("data_version", data_version)
mlflow.log_artifact(annotation_file)

In [6]:
# split train and validation in Training
trainDataFile = "train.data"
trainingData = pd.read_csv(trainDataFile, sep="\t", header=0)
train_features = trainingData.copy()
train_labels = train_features.pop(train_features.columns[0]) # the first column
train_features = np.array(train_features)
train_values, ids, train_labels_indexed = np.unique(train_labels, return_index=True, return_inverse=True)

train_labels_encoded = to_categorical(train_labels_indexed)

In [7]:
model = Sequential([
    Input(shape=len(train_features[0])),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(4, activation='softmax')
])
model.compile(optimizer='adam', loss=tf.keras.losses.categorical_crossentropy,
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               2432      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 4)                 516       
                                                                 
Total params: 19,460
Trainable params: 19,460
Non-trainable params: 0
_________________________________________________________________


2023-03-12 06:24:09.876681: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-12 06:24:09.876825: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-12 06:24:09.888778: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2023-03-12 06:24:09.888798: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to do

In [8]:
model.fit(x=train_features, y=train_labels_encoded,
          validation_split=0.2,
          epochs=100)

Epoch 1/100
50/50 [==============================] - 0s 2ms/step - loss: 18.9892 - accuracy: 0.3682 - val_loss: 3.8924 - val_accuracy: 0.4625
Epoch 2/100
50/50 [==============================] - 0s 913us/step - loss: 5.7932 - accuracy: 0.4715 - val_loss: 1.2931 - val_accuracy: 0.6625
Epoch 3/100
50/50 [==============================] - 0s 835us/step - loss: 2.7909 - accuracy: 0.5429 - val_loss: 0.7081 - val_accuracy: 0.7400
Epoch 4/100
50/50 [==============================] - 0s 838us/step - loss: 1.8179 - accuracy: 0.5473 - val_loss: 0.6511 - val_accuracy: 0.7575
Epoch 5/100
50/50 [==============================] - 0s 738us/step - loss: 1.3338 - accuracy: 0.6043 - val_loss: 0.6149 - val_accuracy: 0.7375
Epoch 6/100
50/50 [==============================] - 0s 691us/step - loss: 1.0584 - accuracy: 0.6381 - val_loss: 0.4974 - val_accuracy: 0.8325
Epoch 7/100
50/50 [==============================] - 0s 748us/step - loss: 0.8441 - accuracy: 0.6963 - val_loss: 0.4378 - val_accuracy: 0.8225


2023/03/12 06:24:57 WARNING mlflow.tensorflow: Failed to infer model signature: could not sample data to infer model signature: tuple index out of range
2023/03/12 06:24:57 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: /tmp/tmpgrz_glyg/model/data/model/assets


INFO:tensorflow:Assets written to: /tmp/tmpgrz_glyg/model/data/model/assets
2023/03/12 06:24:59 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/binf009/anaconda3/envs/PoseDetectorDLCLive_Training/lib/python3.8/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."


In [9]:
# TODO extract to function
testDataFile = "test.data"
testData = pd.read_csv(testDataFile, sep="\t", header=0)
test_features = testData.copy()
test_labels = test_features.pop(test_features.columns[0])  # the first column
test_features = np.array(test_features)
test_values, test_ids, test_labels_indexed = np.unique(test_labels, return_index=True, return_inverse=True)

test_labels_encoded = to_categorical(test_labels_indexed)

In [10]:
model.evaluate(test_features, test_labels_encoded)

16/16 [==============================] - 0s 584us/step - loss: 0.1506 - accuracy: 0.9379


[0.1506328284740448, 0.9378757476806641]

In [12]:
for i in range(10):
    down, sit, stand, unknown  = model.predict(np.array([test_features[i]]), verbose=0)[0]
    print(f"{test_labels[i]}\t- Stand: {stand} Sit: {sit} Down: {down} Unknown: {unknown}")

Unknown	- Stand: 8.62798401612963e-08 Sit: 0.0005128846969455481 Down: 0.0017513869097456336 Unknown: 0.9977356195449829
Down	- Stand: 0.0022722184658050537 Sit: 2.019942080266901e-08 Down: 0.9695284366607666 Unknown: 0.02819937653839588
Stand	- Stand: 0.9910507202148438 Sit: 6.5738590819819365e-06 Down: 1.0985963854182046e-05 Unknown: 0.008931683376431465
Stand	- Stand: 0.8711495995521545 Sit: 0.0008489449392072856 Down: 9.284805742026947e-07 Unknown: 0.12800051271915436
Down	- Stand: 4.167612477345983e-09 Sit: 1.0327091359840779e-07 Down: 0.987862229347229 Unknown: 0.012137604877352715
Down	- Stand: 4.252246734257462e-25 Sit: 3.5408360453774084e-29 Down: 1.0 Unknown: 2.1871350952551438e-08
Unknown	- Stand: 0.7155418992042542 Sit: 5.3770500016980805e-06 Down: 3.971677529079898e-07 Unknown: 0.28445228934288025
Down	- Stand: 0.00016875778965186328 Sit: 1.8438535676068568e-07 Down: 0.9914650321006775 Unknown: 0.008366020396351814
Unknown	- Stand: 1.5187303006314323e-06 Sit: 4.11048306503

In [13]:
# Now export the model to use it after the DLC-live prediction
# Don't export, use the mlflow id for prediction
import os
savedModelDir = os.path.join(os.getcwd(), "model/2/")
tf.keras.models.save_model(model, savedModelDir)

INFO:tensorflow:Assets written to: /home/binf009/projects/PoseDetector/DLCLive/Training/model/2/assets


INFO:tensorflow:Assets written to: /home/binf009/projects/PoseDetector/DLCLive/Training/model/2/assets


In [16]:
# adding model from local path does not work in ui
# until fixed, store model via api
mlflow.keras.log_model(
    model=model,
    artifact_path="keras-model",
    registered_model_name="PoseClassification"
)

2023/03/12 11:25:52 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: /tmp/tmprrig4b96/model/data/model/assets


INFO:tensorflow:Assets written to: /tmp/tmprrig4b96/model/data/model/assets
Registered model 'PoseClassification' already exists. Creating a new version of this model...
2023/03/12 11:25:54 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: PoseClassification, version 1
Created version '1' of model 'PoseClassification'.
